In [2]:
! pip install transformers datasets evaluate

In [ ]:
from huggingface_hub import notebook_login

# notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset

dataset = load_dataset("aslg_pc12")
dataset = dataset["train"].train_test_split(train_size=0.8)

In [40]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

from collections import defaultdict
word_freqs = defaultdict(int)
for i in range(0, len(dataset["train"])):
  for text in dataset["train"][i]["gloss"]:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
  for text in dataset["train"][i]["text"]:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
for i in range(0, len(dataset["test"])):
  for text in dataset["test"][i]["gloss"]:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1
  for text in dataset["test"][i]["text"]:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

char_freqs = defaultdict(int)
subwords_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        # Loop through the subwords of length at least 2
        for j in range(i + 2, len(word) + 1):
            subwords_freqs[word[i:j]] += freq

# Sort subwords by frequency
sorted_subwords = sorted(subwords_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords[:10]

token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}

from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        # This should be properly filled by the previous steps of the loop
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx:end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                # If we have found a better segmentation ending at end_idx, we update
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        # We did not find a tokenization of the word -> unknown
        return ["<unk>"], None

    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start:end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start:end])
    return tokens, score
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += freq * word_loss
    return loss

import copy


def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep tokens of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - model_loss
    return scores
scores = compute_scores(model)

In [41]:
source_lang = "gloss"
target_lang = "text"


def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [42]:
tokenized_data = dataset.map(preprocess_function, batched=True)

  0%|          | 0/71 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [43]:
! pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import evaluate

sacrebleu = evaluate.load("sacrebleu")

In [45]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [46]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/02bc3551ce463fd96bd4c3735822469bace0396d/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
   

In [47]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [48]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.144700,0.744925,49.302200,15.629100
2,0.946500,0.628684,54.423200,15.553600
3,0.881000,0.600445,55.619200,15.553800


Saving model checkpoint to transformer_model/checkpoint-13000
Configuration saved in transformer_model/checkpoint-13000/config.json
Model weights saved in transformer_model/checkpoint-13000/pytorch_model.bin
tokenizer config file saved in transformer_model/checkpoint-13000/tokenizer_config.json
Special tokens file saved in transformer_model/checkpoint-13000/special_tokens_map.json
Deleting older checkpoint [transformer_model/checkpoint-11500] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text, gloss. If text, gloss are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17542
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=13158, training_loss=1.117422225356084, metrics={'train_runtime': 2166.7255, 'train_samples_per_second': 97.153, 'train_steps_per_second': 6.073, 'total_flos': 3200474103152640.0, 'train_loss': 1.117422225356084, 'epoch': 3.0})

In [49]:
dataset["train"][20]

{'gloss': 'X-I WOULD REMIND X-YOU THAT X-WE DO DESC-NOT HAVE DESC-GENERAL DESC-LEGISLATIVE POWER ON EVERY ISSUE .\n',
 'text': 'i would remind you that we do not have general legislative power on every issue .\n'}

In [50]:
text = dataset["train"][20]["gloss"]

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="transformer_model")

prediction = []
answer = []

def testing_data():
  for i in range(20):
    text = dataset["train"][i]["gloss"]
    answer.append(dataset["train"][i]["text"])
    prediction.append(translator(text))

In [52]:
testing_data()
print(prediction)
print(answer)

[[{'translation_text': 'X-I think this is debating that we need to have .'}], [{'translation_text': 'I think it is the same that we have a large area of security .'}], [{'translation_text': 'es ist kein Weg für Kosovo mit Blick auf die SERBIUM X-POSS-Border .'}], [{'translation_text': 'Sehr geehrter Herr Vorsitzender , wir hätten darüber abgestimmt, wen wir das wollen oder nicht .'}], [{'translation_text': 'Diese weltliche Krise ist durch die europäische Integration zerstreut .'}], [{'translation_text': 'GREAT , and we welcome that .'}], [{'translation_text': 'ich ziehe, daß die Abgeordneten auf diesen drei Zielen abstimmen.'}], [{'translation_text': 'PERHAPS Y HAVE NOT read it .'}], [{'translation_text': "i ask you why you can't be more cooperative about the question of DRUZHBA PIPELINE ."}], [{'translation_text': 'Die EU-Erarbeitungshilfe im WATER-Sicherheitsbereich hat sich entschlossen, dies zu verwirklichen .'}], [{'translation_text': 'Der Abbau ist recht erkennbar .'}], [{'transl